In [20]:
!pip3 install fitz
!pip3 install pymupdf
!pip3 install pdfplumber 
!pip3 install pytesseract pdf2image pillow
!brew install poppler

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Downloading https://ghcr.io/v2/homebrew/portable-ruby/portable-ruby/blobs/sha256:e7340e4a1d7cc0f113686e461b93114270848cb14676e9037a1a2ff3b1a0ff32
######################################################################### 100.0%
==> Pouring portable-ruby-3.3.5.arm64_big_sur.bottle.tar.gz
==> Auto-updated Homebrew!
Updated 3 taps (homebrew/services, homebrew/core and homebrew/cask).
==> New Formulae
afl++                      jikken                     paperjam
aicommit                   j

In [21]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF
import numpy as np
import os
import pdfplumber

In [2]:
def extraer_texto_de_pdf(ruta_pdf):
    """
    Extrae el texto de un documento PDF.
    :param ruta_pdf: Ruta del archivo PDF.
    :return: Texto extraído del PDF.
    """
    # Abre el documento PDF
    documento = fitz.open(ruta_pdf)
    texto_completo = ""

    # Itera sobre cada página del documento
    for pagina_num in range(len(documento)):
        pagina = documento[pagina_num]
        texto = pagina.get_text()  # Extrae el texto de la página
        texto_completo += texto

    documento.close()
    return texto_completo


In [1]:
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

# Configura la ruta a Tesseract si es necesario (en Windows)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extraer_texto_de_pdf_con_ocr(ruta_pdf):
    """
    Extrae el texto de un documento PDF utilizando OCR (Tesseract) y pdf2image.
    :param ruta_pdf: Ruta del archivo PDF.
    :return: Texto extraído del PDF.
    """
    texto_completo = ""

    # Convierte el PDF a una lista de imágenes (una imagen por página)
    paginas = convert_from_path(ruta_pdf)

    # Realiza OCR en cada imagen de página
    for pagina in paginas:
        texto = pytesseract.image_to_string(pagina)  # Realiza OCR en la imagen de la página
        texto_completo += texto

    return texto_completo


In [2]:
extraer_texto_de_pdf_con_ocr('/Users/alejandromunoz/Desktop/Proyectos/VK/LLM DOCUMENTOS/libros/Vigencia Ing. Sandro Che - Chimu - Jefe de Sistemas - 29.11.23.pdf')

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [16]:
def crear_df_de_directorio(directorio):
    """
    Lee todos los archivos PDF en un directorio y crea un DataFrame con tres columnas:
    id, titulo (nombre del archivo), texto (contenido extraído).
    :param directorio: Ruta del directorio que contiene los archivos PDF.
    :return: DataFrame con columnas id, titulo, texto.
    """
    archivos = [f for f in os.listdir(directorio) if f.endswith('.pdf')]
    
    datos = []  # Lista para almacenar los datos de los PDFs
    
    for idx, archivo in enumerate(archivos, 1):
        ruta_pdf = os.path.join(directorio, archivo)
        texto = extraer_texto_de_pdf(ruta_pdf)  # Extraemos el texto del PDF
        
        datos.append({
            'id': idx,
            'titulo': archivo,
            'texto': texto
        })
    
    # Crear un DataFrame con los datos recopilados
    df = pd.DataFrame(datos, columns=['id', 'titulo', 'texto'])
    
    return df

In [17]:
directorio = "/Users/alejandromunoz/Desktop/Proyectos/VK/LLM DOCUMENTOS/libros"
df_pdfs = crear_df_de_directorio(directorio)
df_pdfs

,id,titulo,texto
0,1,019-2024-EF.pdf,14 NORMAS LEGALES Sábado 5 de octubre de 2024/...
1,2,Vigencia Ing. Sandro Che - Chimu - Jefe de Sis...,
2,3,000047-2024-DP.pdf,40 NORMAS LEGALES Viernes 4 de octubre de 2024...
3,4,324-2024.pdf,El Peruano / Sábado 5 de octubre de 2024 NORMA...
4,5,307-2024-EF.pdf,El Peruano / Sábado 5 de octubre de 2024 NORMA...
5,6,017-2024-CR.pdf,El Peruano / Sábado 5 de octubre de 2024 NORMA...
6,7,210-2024-PCM.pdf,4 NORMAS LEGALES Sábado 5 de octubre de 2024/ ...
7,8,492-MVES.pdf,54 NORMAS LEGALES Sábado 5 de octubre de 2024/...


In [5]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# Función para limpiar el texto
def limpiar_texto(texto):
    texto = re.sub(r'•|–|-|–|—', '', texto)  # Eliminar viñetas comunes y guiones
    texto = re.sub(r'\n\s*\n', '\n', texto)  # Eliminar saltos de línea excesivos
    texto = re.sub(r'\s+', ' ', texto)       # Reducir múltiples espacios a uno solo
    texto = texto.strip()                    # Eliminar espacios en blanco al inicio y al final
    return texto

# Función para preprocesar el texto (en español)
def preprocesar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúñü ]', '', texto)  # Filtrar caracteres no alfabéticos y tildes
    
    stop_words = set(stopwords.words('spanish'))
    palabras = texto.split()
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stop_words]
    
    lemmatizer = WordNetLemmatizer()  # Nota: Este lematizador es para inglés. Considera spaCy para español.
    palabras_lemmatizadas = [lemmatizer.lemmatize(palabra) for palabra in palabras_filtradas]
    
    texto_procesado = ' '.join(palabras_lemmatizadas)
    
    return texto_procesado

# Aplicar las funciones al DataFrame
df_pdfs['texto'] = df_pdfs['texto'].apply(limpiar_texto)
df_pdfs['texto'] = df_pdfs['texto'].apply(preprocesar_texto)

# Mostrar los primeros registros del DataFrame actualizado
df_pdfs


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alejandromunoz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alejandromunoz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,titulo,texto
0,1,019-2024-EF.pdf,norma legales sábado octubre peruano considera...
1,2,Vigencia Ing. Sandro Che - Chimu - Jefe de Sis...,
2,3,000047-2024-DP.pdf,norma legales viernes octubre peruano organism...
3,4,324-2024.pdf,norma legales sábado octubre peruano carlos al...
4,5,307-2024-EF.pdf,norma legales sábado octubre peruano legislati...
5,6,017-2024-CR.pdf,norma legales sábado octubre peruano planeta v...
6,7,210-2024-PCM.pdf,norma legales sábado octubre peruano poder eje...
7,8,492-MVES.pdf,norma legales sábado octubre peruano decreta a...


In [6]:
df_pdfs.iloc[1]

id                                                        2
titulo    Vigencia Ing. Sandro Che - Chimu - Jefe de Sis...
texto                                                      
Name: 1, dtype: object

In [34]:
pregunta = "Cual es el objetivo de la Ley del Servicio Civil?"

In [23]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection
from pymilvus.exceptions import MilvusException
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd

# Conexión a Milvus
def conectar_milvus(host="localhost", port="19530"):
    connections.connect("default", host=host, port=port)
    print("Conexión exitosa a Milvus")

# Creación de colección
def crear_coleccion(nombre_coleccion, embedding_size):
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=embedding_size),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=4096),
        FieldSchema(name="document_id", dtype=DataType.INT64)
    ]
    schema = CollectionSchema(fields, "Colección para almacenar embeddings, textos y document_id")
    
    try:
        collection = Collection(name=nombre_coleccion, schema=schema)
        print(f"Colección '{nombre_coleccion}' creada con éxito")
    except MilvusException as e:
        print(f"Error al crear la colección: {e}")
        collection = Collection(name=nombre_coleccion)

    index_params = {
        "metric_type": "L2",
        "index_type": "HNSW",
        "params": {"M": 16, "efConstruction": 200}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    print("Índice creado con éxito")

    return collection

# Inserción de embeddings, textos y document_id
def insertar_embeddings_y_textos(collection, chunked_documents, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = []
    texts = []
    document_ids = []

    for doc in chunked_documents:
        text = doc['text']
        document_id = doc['document_id']
        embedding = model.encode(text)
        embeddings.append(embedding)
        texts.append(text)
        document_ids.append(document_id)

    embeddings_flat = [embedding.tolist() for embedding in embeddings]
    data = [embeddings_flat, texts, document_ids]
    
    collection.insert(data)
    print(f"Insertados {len(embeddings)} embeddings en la colección '{collection.name}'")

# Búsqueda de embeddings en Milvus
def buscar_embeddings(collection, embedding_query, top_k=5):
    embedding_query_flat = embedding_query.tolist()
    collection.load()
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    try:
        resultados = collection.search([embedding_query_flat], "embedding", search_params, limit=top_k, output_fields=["id", "text", "document_id"])

        chunks_encontrados = []
        for resultado in resultados:
            for hit in resultado:
                chunk_info = {
                    "id": hit.id,
                    "text": hit.entity.get('text'),
                    "document_id": hit.entity.get('document_id'),
                    "distance": hit.distance
                }
                chunks_encontrados.append(chunk_info)
                print(f"ID: {chunk_info['id']}, Distancia: {chunk_info['distance']}, Texto: {chunk_info['text']}, Document ID: {chunk_info['document_id']}")

        return chunks_encontrados
    except MilvusException as e:
        print(f"Error durante la búsqueda: {e}")
        raise

# Obtener embedding
def get_embedding(text, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    return model.encode(text)

# Buscar otro chunk en la lista `chunked_documents` con el mismo document_id y la palabra "PROBLEM"
def buscar_chunk_con_problema(chunked_documents, document_id):
    for chunk in chunked_documents:
        if chunk['document_id'] == document_id and 'PROBLEM' in chunk['text']:
            return chunk
    return None

# Integración con Milvus usando df_pdfs
if __name__ == "__main__":
    conectar_milvus()

    nombre_coleccion = "embedding_collection"
    embedding_size = 384  # Tamaño del embedding del modelo
    collection = crear_coleccion(nombre_coleccion, embedding_size)

    # df_pdfs es el DataFrame con las columnas id, titulo y texto
    df_pdfs = df_pdfs  # Asegúrate de que df_pdfs esté correctamente definido antes

    # Dividir documentos en chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
    chunked_documents = []
    chunk_unique_id = 0

    for doc_id, row in df_pdfs.iterrows():
        document = row['texto']  # Extraemos el texto desde la columna 'texto'
        chunks = text_splitter.split_text(document)
        for chunk_id, chunk in enumerate(chunks):
            chunked_documents.append({
                "id": chunk_unique_id,
                "document_id": doc_id,  # Usamos el id de df_pdfs como document_id
                "text": chunk
            })
            chunk_unique_id += 1

    # Insertar los embeddings en la colección de Milvus
    insertar_embeddings_y_textos(collection, chunked_documents)

    pregunta = pregunta
    embedding_pregunta = get_embedding(pregunta)

    top_3emb = buscar_embeddings(collection, embedding_pregunta, top_k=3)
    
    
    

Conexión exitosa a Milvus
Colección 'embedding_collection' creada con éxito
Índice creado con éxito


/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Insertados 40 embeddings en la colección 'embedding_collection'
ID: 453091570143461485, Distancia: 0.953639030456543, Texto: norma legales viernes octubre peruano organismos ejecutores despacho presidencial designan jefa oficina general administración despacho presidencial resolución secretaría general n dpsg lima octubre vistos memorando n dpsg fecha octubre secretaría general memorando n dpssg fecha octubre subsecretaría general informes n dpssgorh n dpssgorhadpc fecha octubre informe n dpogaj informe legal n dpogajsss fecha octubre oficina general asesoría jurídica considerando ley n ley servicio civil objetivo establecer régimen único exclusivo persona prestan servicios entidades públicas así aquellas persona encargadas gestión ejercicio potestades prestación servicios cargo artículo ley n ley servicio civil establece servidores civiles entidades públicas clasifican grupos funcionarios públicos directivos públicos servidores civiles carrera servidores actividades complementarias pr

In [36]:
top_3emb[0]

{'id': 453091570143461485,
 'text': 'norma legales viernes octubre peruano organismos ejecutores despacho presidencial designan jefa oficina general administración despacho presidencial resolución secretaría general n dpsg lima octubre vistos memorando n dpsg fecha octubre secretaría general memorando n dpssg fecha octubre subsecretaría general informes n dpssgorh n dpssgorhadpc fecha octubre informe n dpogaj informe legal n dpogajsss fecha octubre oficina general asesoría jurídica considerando ley n ley servicio civil objetivo establecer régimen único exclusivo persona prestan servicios entidades públicas así aquellas persona encargadas gestión ejercicio potestades prestación servicios cargo artículo ley n ley servicio civil establece servidores civiles entidades públicas clasifican grupos funcionarios públicos directivos públicos servidores civiles carrera servidores actividades complementarias precisando cualquiera grupos pueden existir servidores confianza artículo citada ley dispo

In [37]:
df_pdfs

,id,titulo,texto
0,1,019-2024-EF.pdf,norma legales sábado octubre peruano considera...
1,2,000047-2024-DP.pdf,norma legales viernes octubre peruano organism...
2,3,324-2024.pdf,norma legales sábado octubre peruano carlos al...
3,4,307-2024-EF.pdf,norma legales sábado octubre peruano legislati...
4,5,017-2024-CR.pdf,norma legales sábado octubre peruano planeta v...
5,6,210-2024-PCM.pdf,norma legales sábado octubre peruano poder eje...
6,7,492-MVES.pdf,norma legales sábado octubre peruano decreta a...


In [10]:
# Para eliminar la base de datos vectorial
# collection.drop()

In [25]:
top_3emb[0]

{'id': 453091570143461485,
 'text': 'norma legales viernes octubre peruano organismos ejecutores despacho presidencial designan jefa oficina general administración despacho presidencial resolución secretaría general n dpsg lima octubre vistos memorando n dpsg fecha octubre secretaría general memorando n dpssg fecha octubre subsecretaría general informes n dpssgorh n dpssgorhadpc fecha octubre informe n dpogaj informe legal n dpogajsss fecha octubre oficina general asesoría jurídica considerando ley n ley servicio civil objetivo establecer régimen único exclusivo persona prestan servicios entidades públicas así aquellas persona encargadas gestión ejercicio potestades prestación servicios cargo artículo ley n ley servicio civil establece servidores civiles entidades públicas clasifican grupos funcionarios públicos directivos públicos servidores civiles carrera servidores actividades complementarias precisando cualquiera grupos pueden existir servidores confianza artículo citada ley dispo

In [32]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama

# Instanciar el modelo Ollama
llm = Ollama(model="llama3.1:8b")



prompt_template = """
Quiero que respondas a la pregunta que te hago a partir de la información que te estoy dando como contexto

Contexto: {context}

Además, te estoy entregando el nombre del archivo del cual he extraido ese contexto, para que puedas hacer referencia a este al momento de dar tu respuesta. No tienes que acceder al archivo ya que todo lo que necesitas te lo estoy dando en el contexto.

Nombre del Documento: {titulo}

Pregunta: {pregunta}

Quiero que respondas a mi pregunta de manera clara. Primero me das la respuesta y luego me dices de que documento has sacado la información
 
"""

# Función para obtener el resumen utilizando Ollama
def get_summary_ollama(context_text,titulo, pregunta):
    # Crear el prompt final utilizando el contexto
    final_prompt = prompt_template.format(context=context_text, titulo=titulo, pregunta=pregunta)
    
    # Generar la respuesta utilizando el modelo Ollama
    response = llm.invoke(final_prompt)
    
    return response

# Función para procesar la respuesta de Ollama
def get_answer_ollama(context_text, titulo, pregunta):
    # Obtener la respuesta de Ollama
    summary = get_summary_ollama(context_text,titulo, pregunta)
    return summary


context_text = top_3emb[0]['text']
id_documento = top_3emb[0]['document_id']
titulo_documento = df_pdfs['titulo'].iloc[int(id_documento)]

# Pregunta de ejemplo
pregunta = pregunta

# Obtener y mostrar la respuesta de Ollama
print(pregunta)
print()
respuesta = get_answer_ollama(context_text,titulo_documento, pregunta)
print(respuesta)


Cual es el objetivo de la Ley del Servicio Civil?

La respuesta a tu pregunta es:

El objetivo de la Ley del Servicio Civil es establecer un régimen único y exclusivo para las personas que prestan servicios en entidades públicas, así como aquellas encargadas de la gestión y ejercicio de potestades y prestación de servicios.

La información que te proporciono se encuentra en el documento 000047-2024-DP.pdf.
